# Live Camera

In [4]:
import tensorflow_hub as hub
import cv2
import numpy
import tensorflow as tf
import pandas as pd

# Carregar modelos
detector = hub.load("https://tfhub.dev/tensorflow/efficientdet/lite2/detection/1")
labels = pd.read_csv('labels.csv',sep=';',index_col='ID')
labels = labels['OBJECT (2017 REL.)']

cap = cv2.VideoCapture(0)
#cap = cv2.cvtColor(cap, cv2.COLOR_BGR2RGB)

width = 448
height = 448

while(True):
    #Capture frame-by-frame
    ret, frame = cap.read()
    
    #Resize to respect the input_shape
    inp = cv2.resize(frame, (width , height ))

    #Convert img to RGB


    rgb = cv2.cvtColor(inp, cv2.COLOR_BGR2RGB)

    #Is optional but i recommend (float convertion and convert img to tensor image)
    rgb_tensor = tf.convert_to_tensor(rgb, dtype=tf.uint8)

    #Add dims to rgb_tensor
    rgb_tensor = tf.expand_dims(rgb_tensor , 0)

    boxes, scores, classes, num_detections = detector(rgb_tensor)
 
    pred_labels = classes.numpy().astype('int')[0]
    pred_boxes = boxes.numpy()[0].astype('int')
    pred_scores = scores.numpy()[0]

    # Extract only boxes and labels corresponding to 'person' class
    person_boxes = []
    person_labels = []
    for score, (ymin,xmin,ymax,xmax), label in zip(pred_scores, pred_boxes, pred_labels):
        if score < 0.5:
            continue
        if labels[label] == 'person':
            person_boxes.append((ymin, xmin, ymax, xmax))
            person_labels.append(labels[label])

    # loop throughout the person detections and place a box around it
    for box, label in zip(person_boxes, person_labels):
        ymin, xmin, ymax, xmax = box
        img_boxes = cv2.rectangle(inp, (xmin, ymin), (xmax, ymax), (0, 255, 0), 1)      
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img_boxes, label, (xmin, ymax-10), font, 0.5, (255, 0, 0), 1, cv2.LINE_AA)

    
    #Display the resulting frame
    cv2.imshow('black and white', img_boxes)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

In [3]:
import tensorflow_hub as hub
import cv2
import numpy
import tensorflow as tf
import pandas as pd

# Carregar modelos
detector = hub.load(r"D:\Python\models\Tf_models")
labels = pd.read_csv('labels.csv',sep=';',index_col='ID')
labels = labels['OBJECT (2017 REL.)']

#labels = ['person']
cap = cv2.VideoCapture('D:\Python\pexels-pablo-cortés-alegría-3551155-3840x2160-60fps.mp4')
#cap = cv2.VideoCapture(0)
width = 448
height = 448
# Define the central region of interest (ROI) by specifying its top-left and bottom-right corners
roi_top_left = (44, 44)
roi_bottom_right = (402, 402)

while(True):
    #Capture frame-by-frame
    ret, frame = cap.read()
    
    #Resize to respect the input_shape
    inp = cv2.resize(frame, (width , height ))

    #Convert img to RGB
    rgb = cv2.cvtColor(inp, cv2.COLOR_BGR2RGB)

    #Is optional but i recommend (float convertion and convert img to tensor image)
    rgb_tensor = tf.convert_to_tensor(rgb, dtype=tf.uint8)

    #Add dims to rgb_tensor
    rgb_tensor = tf.expand_dims(rgb_tensor , 0)

    boxes, scores, classes, num_detections = detector(rgb_tensor)
    #print(classes)
    pred_labels = classes.numpy().astype('int')[0]
    pred_boxes = boxes.numpy()[0].astype('int')
    pred_scores = scores.numpy()[0]
    #print(pred_labels)
    # Extract only boxes and labels corresponding to 'person' class
    person_boxes = []
    person_labels = []
    for score, (ymin,xmin,ymax,xmax), label in zip(pred_scores, pred_boxes, pred_labels):
        if score < 0.5:
            continue
        if labels[label] == 'person':
            person_boxes.append((ymin, xmin, ymax, xmax))
            person_labels.append(labels[label])

    # loop throughout the person detections and place a box around it
    for box, label in zip(person_boxes, person_labels):
        ymin, xmin, ymax, xmax = box
        #print(xmin,ymin)
        #print(xmax,ymax)
        # Check if the person box is within the central ROI
        if (xmin >= roi_top_left[0] and xmin <= roi_bottom_right[0] and 
            ymin >= roi_top_left[1] and ymin <= roi_bottom_right[1]) or \
            (xmax >= roi_top_left[0] and xmax <= roi_bottom_right[0] and 
            ymax >= roi_top_left[1] and ymax <= roi_bottom_right[1]) or \
            (xmin >= roi_top_left[0] and xmin <= roi_bottom_right[0] and 
            ymax >= roi_top_left[1] and ymax <= roi_bottom_right[1]) or \
            (xmax >= roi_top_left[0] and xmax <= roi_bottom_right[0] and 
            ymin >= roi_top_left[1] and ymin <= roi_bottom_right[1]):           
            # Draw the person box in green if it's within the ROI
            img_boxes = cv2.rectangle(inp, (xmin, ymin), (xmax, ymax), (0, 255, 0), 1)      
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(img_boxes, label, (xmin, ymax-10), font, 0.5, (255, 0, 0), 1, cv2.LINE_AA)
        
        else:
            continue
            # Draw the person box in red if it's outside the ROI
            #img_boxes = cv2.rectangle(inp, (xmin, ymin), (xmax, ymax), (0, 0, 255), 1)      
            #font = cv2.FONT_HERSHEY_SIMPLEX
            #cv2.putText(img_boxes, label, (xmin, ymax-10), font, 0.5, (255, 0, 0), 1, cv2.LINE_AA)

    # Draw the ROI on the frame
    roi_frame = cv2.rectangle(inp, roi_top_left, roi_bottom_right, (255, 0, 0), 1)

    # Display the resulting frame
    cv2.imshow('frame',roi_frame)

    if cv2.waitKey(2) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()


In [6]:
import tensorflow as tf
import cv2
import numpy as np

# Load the TFLite model
def load_model(model_path):
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    return interpreter

# Preprocess the input image
def preprocess_image(image, size):
    image = cv2.resize(image, size)
    image = tf.convert_to_tensor(image, dtype=tf.uint8)
    return image[tf.newaxis, ...]

# Run inference on the preprocessed image
def run_inference(interpreter, image):
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    interpreter.set_tensor(input_details[0]["index"], image)
    interpreter.invoke()
    boxes = interpreter.get_tensor(output_details[0]["index"])
    classes = interpreter.get_tensor(output_details[1]["index"])
    scores = interpreter.get_tensor(output_details[2]["index"])

    return boxes, classes, scores
def main():
    model_path = r"D:\Python\models\model_lite_2.tflite"
    size = (448,448)  # EfficientDet Lite1 input size is 320x320
    width = 448
    height = 448
    # Load the model
    interpreter = load_model(model_path)

    # Open webcam
    #cap = cv2.VideoCapture('D:\Python\pexels-pablo-cortés-alegría-3551155-3840x2160-60fps.mp4')
    cap = cv2.VideoCapture(0)
    roi_top_left = (44, 44)
    roi_bottom_right = (402,402)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        
        inp = cv2.resize(frame, (width , height ))
        # Preprocess the input image
        preprocessed_image = preprocess_image(inp, size)
        
        # Run inference
        boxes, classes, scores = run_inference(interpreter, preprocessed_image)

        # Display the results
        for i in range(len(boxes[0])):
            if scores[0][i] > 0.5 and int(classes[0][i]) == 0:  # Person class has index 0
                ymin, xmin, ymax, xmax = boxes[0][i]
                
                xmin *= preprocessed_image.shape[1]
                xmax *= preprocessed_image.shape[1]
                ymin *= preprocessed_image.shape[1]
                ymax *= preprocessed_image.shape[1]

                # Check if the person is within the ROI
                if (xmin >= roi_top_left[0] and xmin <= roi_bottom_right[0] and 
                    ymin >= roi_top_left[1] and ymin <= roi_bottom_right[1]) or \
                    (xmax >= roi_top_left[0] and xmax <= roi_bottom_right[0] and 
                    ymax >= roi_top_left[1] and ymax <= roi_bottom_right[1]) or \
                    (xmin >= roi_top_left[0] and xmin <= roi_bottom_right[0] and 
                    ymax >= roi_top_left[1] and ymax <= roi_bottom_right[1]) or \
                    (xmax >= roi_top_left[0] and xmax <= roi_bottom_right[0] and 
                    ymin >= roi_top_left[1] and ymin <= roi_bottom_right[1]):
                    cv2.rectangle(inp, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 2)
                    cv2.putText(inp, f"Person, Score: {scores[0][i]:.2f}", (int(xmin), int(ymin) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Draw the ROI on the frame
        roi_frame = cv2.rectangle(inp, roi_top_left, roi_bottom_right, (255, 0, 0), 1)
        # print(roi_frame.shape)
        cv2.imshow("Output",roi_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
            break

    cap.release()
    cv2.destroyAllWindows()

main()